In [6]:
#Database class to handle all PostgreSQL database connectivity and queries

import psycopg2
import psycopg2.extras

class Database(object):
    
    def __init__(self, debug=False):

        try:
           self.con = psycopg2.connect(database='MSA8010', user='postgres', password='password') 
           self.con.autocommit = True
           self.cur = self.con.cursor(cursor_factory=psycopg2.extras.DictCursor)
           
           if debug:
               self.cur.execute('SELECT version()')          
               print self.cur.fetchone()
    
        except:
            print "Error: unable to connect to database"
            return False


    def read(self, sql, params=None, returnAll=True):

        try:
            #execute a sql select statement and return one or all rows        
            self.cur.execute(sql, params)
            if (returnAll):
                return self.cur.fetchall()
            else:
                return self.cur.fetchone()
        except:
            print "Database error"
            return False
           
           
    def execute(self, sql, params):
        try:
            #Execute a sql command (insert, update, delete)
            self.cur.execute(sql, params)
            return 
        
        except:
            print "Database error"
            return False
       
               
    def close(self):       
        # disconnect from server
        self.cur.close()
        

db=Database(True)

['PostgreSQL 9.3.4, compiled by Visual C++ build 1600, 64-bit']


In [10]:
# This program reads and parses all of the log files in a directory
# It stores unique visits to a database (unique IP address and user agent)
# It also calculates hourly total page loads and saves the totals to a database table

import os
import sys
import datetime

#create second database cursor for updating the records 
import DatabaseClass

#create database connection
db=DatabaseClass.Database()

try:
    #create list to hold hourly counts of activity
    hourCount=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

    #Count only UNIQUE visitors
    #create dictionary to hold processed unique visitors
    IPs={}  
    
    #pre-load dictionary with IPs already processed
    rows=db.read("select userip, useragent from logs")
    for row in rows:
        uniqueVisitor = row[0] + row[1]
        IPs[uniqueVisitor] = 1
        
    print len(IPs), "unique IP + user agents were preloaded."
    
    

    fileCount=0
    sourcePath="c:\\temp\\MSA8010\\test\\"
    lst = os.listdir(sourcePath)
    
    #loop through all files in the directory
    for sourceFile in lst:
        print sourceFile
        sourceFilePath=sourcePath+sourceFile
        if os.path.isdir(sourceFilePath):       #skip subdirectories
            continue
      
        print "Processing",sourceFile
        f = open(sourceFilePath, 'r')           #open the file
        fileCount += 1

        #keep counts of lines processed        
        count=0
        skipped=0       #skip over duplicates
        inserted=0      #number of inserts into the database

        #initialize list of hours in a day to hold activity during that hour        
        for hr in range(0,24):
            hourCount[hr]=0
            
        while True:
            line=f.readline()
            if line =='':       #break on end of file
                break
            count += 1          #line count
    
            if count %10000 == 0:
                print "COUNT=",count
    
            if line[0] == '#':  #skip comment lines
                continue
        
            d=line.split()      #log file fields are space delimeted
            
            hour=int(d[1][0:2])     #record hourly activity count
            hourCount[hour] += 1
            
            
            #we want only unique vistors
            #record entries from the same IP and same user address only once 
            #create unique key from user's IP address and user agent string
            uniqueVisitor = d[8]+d[9]   
            if uniqueVisitor in IPs:
                #print IPs
                #print d[8],"exists. Skipping"
                skipped += 1
                continue

            #cur.execute("""INSERT INTO logs (LogDate,LogTime,UserIP,UserAgent,SourceFile)
            #            VALUES (%s,%s,%s,%s,%s)""", \
            #            (d[0],d[1],d[8],d[9],sourceFile))
            
            inserted += 1
            IPs[uniqueVisitor] = "1"    

        #determine day of week for this log file. Mon=0, Sun=6
        year, month, day = (int(x) for x in d[0].split('-'))    
        dow = datetime.date(year, month, day).weekday()
        
        #record hourly counts for this day
        for hr in range(0, 24):
            if hr < 10 :
                hour = '0'+str(hr)
            else:
                hour = str(hr)                
            db.execute("""INSERT INTO activity VALUES (%s,%s,%s,%s)""", (d[0],dow,hour,hourCount[hr]))
        
        f.close()       #close file
        
        print "Processed", count, "log enries"
        print skipped," duplicates were skipped"
        print inserted,"entries were inserted into the database"
        
    
except IOError:
    print 'Error accessing files'
    sys.exit(1)

finally:

    db.close()
    
    print fileCount, "files were processed"


241148 unique IP + user agents were preloaded.
0 files were processed


In [7]:
# This program scrapes IP address locations from a web site http://whatsmyip.com
# It uses regular expressions to locate the state in the hmtl page that is returned
# There is a limit of 50 before the host locks us out.

import re
import urllib2 


#create second database cursor for updating the records 
import DatabaseClass

#create database connection
db=DatabaseClass.Database()

class CrawlIP:

    def getState(self, ip):
        host="http://whatismyipaddress.com/ip/"
        
        req = urllib2.Request(host+ip, headers={ 'User-Agent': 'Mozilla/5.0' })
        html = urllib2.urlopen(req).read()

        #print html
        
        #get string with any number of characters (using .*?) after State/region:</th></td>
        # .* means any char string and ? means non-greedy (stop at first match)
        match= re.search("State/Region:</th><td>.*?</td></tr>",html)        
        if match:
            loc = match.group(0)
            loc = re.sub('State/Region:</th><td>','',loc) 
            state = re.sub('</td></tr>','',loc)
        else:
            state = "Not Found"
        return state


demo=True

ip=CrawlIP()

if demo:
    ip1="131.96.210.58"
    ip2="66.249.79.84"
            
    print ip1, ip.getState(ip1)
    print ip2, ip.getState(ip2)
    count=2
    
else:    
    rows=db.read("select pkey, userIP from logs where state is null limit 100")
    
    count=0
    for row in rows:
        count += 1
        state=ip.getState(row[1])
        print count, row[0], row[1], state
        db.execute("update logs set state=%s where pkey=%s", (state, row[0]))    

print "Done.", count, "records updated"
db.close()



131.96.210.58 Georgia
66.249.79.84 California
Done. 2 records updated


In [9]:
# This program determines the state of the user's IP address from a databse of IPs
# We used this database when web scraping the data from http://whatsmyip.com stopped working
# because we sent too many requests

#create second database cursor for updating the records 
import DatabaseClass

#create database connection
db=DatabaseClass.Database()

#get a subset of records where the state has not yet been determined 
rows=db.read("select pkey, userIP from logs where state is null order by pkey desc limit 100000")

count=0
for row in rows:
    count += 1    
    getState=db.read("select city from ip_addresses where %s >=startip and %s <=endip order by startip desc limit 1",(row[1], row[1]), False)

    if getState==None:    
        state="unknown"
    else:
        state=getState[0]
      
    print count, row[0],row[1],state
    
    db.execute("update logs set state=%s where pkey=%s", (state, row[0]))    

print count, "records updated"
db.close()


0 records updated


In [8]:
# This program uses regular expressions to determine the operating system and
# browser type from the user agent string reported in the logs

import re

#create second database cursor for updating the records 
import DatabaseClass

#create database connection
db=DatabaseClass.Database()

#get a subset of records where the state has not yet been determined 
rows=db.read("select pkey, useragent from logs where browsername is null")

count=0
emptyCount=0

for row in rows:
    count += 1    
    #print count, row[0], row[1]

    os=''
    browser=''
    
    pkey=row[0]
    ua=row[1]
    
    match = re.search("MSIE.*?;", ua)   #Internet Explorer    
    if match:
        ver = match.group(0)
        ver = re.sub('MSIE.','', ver) 
        ver = re.sub(';','', ver)
        browser = 'IE '+ver
    else:
        match = re.search("Trident.*?;", ua)    #also Internet Explorer
        if match:
            ver = match.group(0)
            ver = re.sub('Trident.','', ver) 
            ver = re.sub(';','', ver)
            browser = 'IE '+ver
        
        else:            
            match = re.search("Firefox",ua)      
            if match:
                browser = 'Firefox'
            else:
                match = re.search("Chrome",ua)      
                if match:
                    browser='Chrome'
                else:
                    match = re.search("Safari",ua)      
                    if match:
                        browser='Safari'
                    else:
                        browser='other'
                    
                
            
    #look for Windows - string may end with either ; or )
    match = re.search("Windows.NT.*?[;)]", ua)  
    if match:
        #print match.group(0)
        
        #Determine Windows version
        ver = match.group(0)
        ver = re.sub('Windows.','', ver) 
        ver = re.sub('[;)]','', ver)
        
        os='Windows '
        if ver == 'NT+6.1': 
            os += '7'
        elif ver == 'NT+6.2' or ver == 'NT+6.3':
            os += '8'
        elif ver == 'NT+6.0':
            os += 'Vista'
        elif ver == 'NT+10.0':
            os += '10'
        elif ver == 'NT+5.1' or ver == 'XP':
            os += 'XP'
        elif ver.startswith('Phone'):
            os += 'Phone'
    
    else:
        match = re.search("Android", ua)  
        if match:
            os='Android'
            
        else:
            match1 = re.search("iPhone", ua)
            match2 = re.search("iPad", ua)

            #Darwin is also used for iOS and Mac desktops    
            darwin = re.search("Darwin", ua)
            if darwin:
                browser='Safari'
            mobileSafari =re.search("MobileSafari", ua)
            if mobileSafari:
                browser='Safari'
                
            if match1 or match2 or (darwin and mobileSafari):
                os='iOS'
                if browser=='':
                    browser = 'Safari'
            else:
                match = re.search("Macintosh", ua)
                if match or (darwin and not mobileSafari):
                    os = 'Mac Desktop'
                else:
                    os = 'other'
                
    #print ua
    if browser=='' or os=='':
        emptyCount += 1
        print emptyCount, pkey, "b=",browser, "os=",os, ua   
    
    db.execute("update logs set browsername=%s, browseros=%s where pkey=%s", (browser, os, pkey))    
            
print "Done.", count, "records were processed.", emptyCount, "could not be processed."  
db.close()
              


Done. 0 records were processed. 0 could not be processed.


In [11]:
#This program performs the regression calculations on trends and creates the plots

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import scipy.stats

#create second database cursor for updating the records 
import DatabaseClass

#create database connection
db=DatabaseClass.Database(True)

def createSeries(resultSet):
    #create dictionary to hold the data pairs {date:hits} in a database result set
    d={}
    for row in resultSet:
        d[row[0]] = row[1]
    return pd.Series(d)
    
#create time series plot of unique user visits per day
#get a subset of records where the state has not yet been determined 
rows=db.read("select logdate, sum(hits) as hits from activity group by logdate order by logdate")

s=createSeries(rows)
s.plot()
print s.describe()



#day of week analysis unsing dataFrame JOIN
rows=db.read("select dayofweek,sum(hits) from activity group by dayofweek")
dow=createSeries(rows)
dowDF=pd.DataFrame(dow, columns=["Unique_Visitors"])

#create labels for day of week values and join the data frames
days=['Mon','Tues','Wed','Thurs','Fri','Sat','Sun']
daysDF=pd.DataFrame(days, columns=["Day"])

uniqueVisitorDays = dowDF.join(daysDF)  #JOIN two data frames
uniqueVisitorDays=uniqueVisitorDays.sort_values("Unique_Visitors", ascending=0)
print
print uniqueVisitorDays

#create labels
labels=[]
fractions=[]
for i in range(7):
    j=i+1
    labels.append(uniqueVisitorDays[i:j].Day.item())

#generate pie chart
uniqueVisitorDays.plot(kind="pie", autopct='%.2f%%', subplots=True, labels=labels, \
        colors=('g','r','b','m','c','y','w'))


#Mobile user analysis

#create time series plot of unique visitors per browser type per month
dfData={}

rows=db.read("select date_part('week',logdate) as week, count(browseros) from logs where browseros='Android' group by date_part('week',logdate),browseros order by date_part('week',logdate)")
dfData['Android'] = createSeries(rows)

rows=db.read("select date_part('week',logdate) as week, count(browseros) from logs where browseros='iOS' group by date_part('week',logdate),browseros order by date_part('week',logdate)")
dfData['iOS'] = createSeries(rows)

rows=db.read("select date_part('week',logdate) as week, count(browseros) from logs where browseros='Windows Phone' group by date_part('week',logdate),browseros order by date_part('week',logdate)")
dfData['Windows_Phone'] = createSeries(rows)

rows=db.read("select date_part('week',logdate) as week, count(browseros) from logs where browseros in ('iOS','Android','Windows Phone') group by date_part('week',logdate) order by date_part('week',logdate)")
allMobile=rows      #save data for linear regression

rows=db.read("select date_part('week',logdate) as week, count(browseros) from logs group by date_part('week',logdate) order by date_part('week',logdate)")
dfData['All_Visitors'] = createSeries(rows)
allTraffic=rows     #save data for linear regression

#sum all mobile traffic counts
dfData['Mobile_Total'] =dfData['Android']+dfData['iOS']+dfData['Windows_Phone']

df=pd.DataFrame(dfData)
df=df.fillna(value=0)       #replace missing data with 0
df.plot()

print df.describe()


#linear regession to find best fit line for all visitors
y=[allTraffic[i][1] for i in range(len(allTraffic))]
x=[allTraffic[i][0] for i in range(len(allTraffic))]

slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x,y)
print
print "All Visitors"
print "slope=", slope
print "intercept=", intercept
print "R-squared=", r_value**2, "standard error=",std_err

x=np.array(x)
y=np.array(y)
regressionLine=slope*x + intercept
df['All_Regression']=regressionLine

#linear regession to find best fit line for mobile users
y=[allMobile[i][1] for i in range(len(allMobile))]
x=[allMobile[i][0] for i in range(len(allMobile))]

slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x,y)
print
print "All Mobile"
print "slope=", slope
print "intercept=", intercept
print "R-squared=", r_value**2, "standard error=",std_err

x=np.array(x)
y=np.array(y)
regressionLine=slope*x + intercept
df['Mobile_Regression']=regressionLine

df.plot(); 


#Analysis of IE6 browsers and Windows XP users
#create time series plot of unique visitors per browser type per month
xp={}

rows=db.read("select date_part('week',logdate) as week, count(browseros) from logs where browsername like 'IE 6%' or (browseros='Windows XP' and browsername like 'IE%') group by date_part('week',logdate) order by date_part('week',logdate)")
xpTraffic=rows      #save data for linear regression
xp['XP'] = createSeries(rows)
xp=pd.DataFrame(xp)
xp=xp.fillna(value=0)       #replace missing data with 0

#linear regession to find best fit line for all visitors
y=[xpTraffic[i][1] for i in range(len(xpTraffic))]
x=[xpTraffic[i][0] for i in range(len(xpTraffic))]

slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x,y)
print
print "XP Browsers"
print "slope=", slope
print "intercept=", intercept
print "R-squared=", r_value**2, "standard error=",std_err

x=np.array(x)
y=np.array(y)
regressionLine=slope*x + intercept
xp['Regression']=regressionLine

xp.plot()


#Create U.S. map of user activity from IP address locations
#matplotlib inline
from collections import defaultdict
import json

from matplotlib import rcParams
import matplotlib.cm as cm
import matplotlib as mpl

#state map of users
states_abbrev = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

#adapted from  https://github.com/dataiap/dataiap/blob/master/resources/util/map_util.py
#load in state geometry
state2poly = defaultdict(list)

data = json.load(file("us-states.json"))

for f in data['features']:
    state = states_abbrev[f['abbrev']]
    geo = f['geometry']
    if geo['type'] == 'Polygon':
        for coords in geo['coordinates']:
            state2poly[state].append(coords)
    elif geo['type'] == 'MultiPolygon':
        for polygon in geo['coordinates']:
            state2poly[state].extend(polygon)

# Look at this link that describes the * and ** for Python arguments
# https://docs.python.org/dev/tutorial/controlflow.html#more-on-defining-functions
# In brief, * and ** for accepting arbitrary number of arguments. * as tuples and ** as dictionary 
            
def draw_state(plot, stateid, **kwargs):
    for polygon in state2poly[stateid]:
        xs, ys = zip(*polygon)
        plot.fill(xs, ys, **kwargs)


def make_map(states, label):
    fig = plt.figure(figsize=(12, 9))
    ax = plt.gca()

    #cmap = cm.binary
    #cmap = cm.copper
    cmap = cm.Reds
    vmin, vmax = 0, states.max().item()//1000
    norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)
    
    for state in states_abbrev.values():
        color = cmap(norm(states.ix[state].item()//1000))
        draw_state(ax, state, color = color, ec='k')

        
    #add an inset colorbar
    ax1 = fig.add_axes([0.45, 0.70, 0.4, 0.02])    
    mpl.colorbar.ColorbarBase(ax1, cmap=cmap, norm=norm, orientation='horizontal')
    ax1.set_title(label)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlim(-180, -60)
    ax.set_ylim(15, 75)
    return ax

rows=db.read("select trim(state),count(state) as unique_visitors from logs group by state order by state")
stateDataSeries=createSeries(rows)
statesDF=pd.DataFrame(stateDataSeries)
make_map(statesDF,"Unique Visitor Count by State (1,000's)")

db.close()

['PostgreSQL 9.3.4, compiled by Visual C++ build 1600, 64-bit']


AttributeError: 'Axes' object has no attribute 'rowNum'